In [1]:
import pandas as pd
import numpy as np
import statistics
import math
import pyblock

# Load data
df = pd.read_csv("results3.txt", sep=' ')

# from kcal/mol to ev/formate 
def apply_scaling(row):
    if '_full_' in row['model']:
        return row['energy'] * 0.0433641153087705 / 36
    elif '_half_' in row['model']:
        return row['energy'] * 0.0433641153087705 / 18
    else:
        return row['energy']

# Rename columns
df.columns = ['model', 'timestep', 'energy']

df['energy'] = df.apply(apply_scaling, axis=1)

# Filter models ending with *_300.0txt
df = df[df['model'].str.endswith('300.0.txt')]

# Convert timestep to nanoseconds
df['timestep'] = round(df['timestep'] * 0.5e-6, 1)

# Group by model and aggregate energy values into lists
df = df.groupby(['model']).agg(list).reset_index()

# Calculate mean and standard deviation for the last 50 energy values
df['mean_energy'] = df['energy'].apply(lambda x: statistics.mean(x))
df['std_energy'] = df['energy'].apply(lambda x: statistics.stdev(x))
df['variance'] = df['energy'].apply(lambda x: statistics.variance(x))

# Values to be used in the loop
values = [9, 17, 25, 33, 41, 49, 57]

# Function to compute block averaged mean and error
def compute_block_averaged_results(energy_values):
    if len(energy_values) <= 80:
        return np.nan, np.nan  # Not enough data to process
    
    energy_array = np.array(energy_values[:-50])
    reblock_data = pyblock.blocking.reblock(energy_array)
    
    if not reblock_data:
        return np.nan, np.nan  # Empty reblock data
    
    opt = pyblock.blocking.find_optimal_block(len(energy_array), reblock_data)
    
    if np.isnan(opt[0]) or int(opt[0]) >= len(reblock_data):
        return np.nan, np.nan  # Invalid optimal block size
    
    block_mean = reblock_data[int(opt[0])].mean
    block_error = reblock_data[int(opt[0])].std_err
    return block_mean, block_error

# Compute block averaged mean and error for each model
df['block_mean'] = df['energy'].apply(lambda x: compute_block_averaged_results(x)[0])
df['block_error'] = df['energy'].apply(lambda x: compute_block_averaged_results(x)[1])

# Initialize lists to store the results
models = []
mean_energy_differences = []
std_energy_differences = []

# Values to be used in the loop
values = [9, 17, 25, 33, 41, 49, 57]

# Loop through the values and calculate the mean and standard deviation differences
for value in values:
    tet_full_model = f'{value}_tet_full_300.0.txt'
    int_full_model = f'{value}_int_full_300.0.txt'
    tet_half_model = f'{value}_tet_half_300.0.txt'
    int_half_model = f'{value}_int_half_300.0.txt'
    
    # Check if the models exist in the DataFrame before calculating the differences
    if tet_full_model in df['model'].values and int_full_model in df['model'].values:
        mean_energy_full_diff = (df[df['model'] == tet_full_model]['block_mean'].values[0] - 
                                 df[df['model'] == int_full_model]['block_mean'].values[0])
        
        # Standard error of the higher resulting value
        tet_full_error = df[df['model'] == tet_full_model]['block_error'].values[0]
        int_full_error = df[df['model'] == int_full_model]['block_error'].values[0]
        std_full = max(tet_full_error, int_full_error)  
        models.append(f'{tet_full_model} - {int_full_model}')
        mean_energy_differences.append(mean_energy_full_diff)
        std_energy_differences.append(std_full)
    
    if tet_half_model in df['model'].values and int_half_model in df['model'].values:
        mean_energy_half_diff = (df[df['model'] == tet_half_model]['block_mean'].values[0] - 
                                 df[df['model'] == int_half_model]['block_mean'].values[0]) 
        
        # Standard error of the higher resulting value
        tet_half_error = df[df['model'] == tet_half_model]['block_error'].values[0]
        int_half_error = df[df['model'] == int_half_model]['block_error'].values[0]
        std_half = max(tet_half_error, int_half_error) 
        
        models.append(f'{tet_half_model} - {int_half_model}')
        mean_energy_differences.append(mean_energy_half_diff)
        std_energy_differences.append(std_half)

# Create a new DataFrame with the results
diff_df = pd.DataFrame({
    'model': models,
    'mean_energy_difference': mean_energy_differences,
    'std_energy_difference': std_energy_differences
})

print(diff_df)


                                            model  mean_energy_difference  \
0     9_tet_full_300.0.txt - 9_int_full_300.0.txt                0.012297   
1     9_tet_half_300.0.txt - 9_int_half_300.0.txt                0.174812   
2   17_tet_full_300.0.txt - 17_int_full_300.0.txt               -0.006123   
3   17_tet_half_300.0.txt - 17_int_half_300.0.txt                0.149917   
4   25_tet_full_300.0.txt - 25_int_full_300.0.txt               -0.007679   
5   25_tet_half_300.0.txt - 25_int_half_300.0.txt                0.130695   
6   33_tet_full_300.0.txt - 33_int_full_300.0.txt               -0.015910   
7   33_tet_half_300.0.txt - 33_int_half_300.0.txt                0.187911   
8   41_tet_full_300.0.txt - 41_int_full_300.0.txt                0.018972   
9   41_tet_half_300.0.txt - 41_int_half_300.0.txt                0.166932   
10  49_tet_full_300.0.txt - 49_int_full_300.0.txt               -0.010741   
11  49_tet_half_300.0.txt - 49_int_half_300.0.txt                0.118448   

In [ ]:
df

In [ ]:
# Function to compute block averaged mean and error
def compute_block_averaged_results(energy_values):
    if len(energy_values) <= 80:
        return np.nan, np.nan  # Not enough data to process
    
    energy_array = np.array(energy_values)
    reblock_data = pyblock.blocking.reblock(energy_array)
    
    if not reblock_data:
        return np.nan, np.nan  # Empty reblock data
    
    opt = pyblock.blocking.find_optimal_block(len(energy_array), reblock_data)
    
    if np.isnan(opt[0]) or int(opt[0]) >= len(reblock_data):
        return np.nan, np.nan  # Invalid optimal block size
    
    block_mean = reblock_data[int(opt[0])].mean
    block_error = reblock_data[int(opt[0])].std_err
    return block_mean, block_error

# Compute block averaged mean and error for each model
df['block_mean'] = df['energy'].apply(lambda x: compute_block_averaged_results(x)[0])
df['block_error'] = df['energy'].apply(lambda x: compute_block_averaged_results(x)[1])

df

In [ ]:
energy_values_int = df.loc[df['model'] == '17_int_full_300.0.txt', 'energy'].values[0]
energy_values_tet = df.loc[df['model'] == '17_tet_full_300.0.txt', 'energy'].values[0]

energy_array_int = np.array(energy_values_int)
energy_array_tet = np.array(energy_values_tet)

In [ ]:
energy_values_int = df.loc[df['model'] == '17_int_full_300.0.txt', 'energy'].values[0]

energy_array = np.array(energy_values_int[:-50])

reblock_data = pyblock.blocking.reblock(energy_array)
for reblock_iter in reblock_data:
    print(reblock_iter)

opt = pyblock.blocking.find_optimal_block(len(energy_array[:-50]), reblock_data)

print(opt)
print(reblock_data[opt[0]].mean, reblock_data[opt[0]].std_err)

In [ ]:
energy_array = np.array(energy_values_int)

reblock_data = pyblock.blocking.reblock(energy_array)
for reblock_iter in reblock_data:
    print(reblock_iter)

opt = pyblock.blocking.find_optimal_block(len(energy_array), reblock_data)
print(opt)
print(reblock_data[opt[0]])

In [ ]:
energy_array = np.array(energy_values_int)

reblock_data = pyblock.blocking.reblock(energy_array[:-20])
for reblock_iter in reblock_data:
    print(reblock_iter)

opt = pyblock.blocking.find_optimal_block(len(energy_array), reblock_data)
print(opt)
print(reblock_data[opt[0]])

In [ ]:
pyblock.error.subtraction(energy_values_int,energy_array_tet,len(energy_array))

In [ ]:
energy_values_int = df.loc[df['model'] == '17_int_full_.0.txt', 'energy'].values[0]

energy_array_tet = np.array(energy_values_tet)

import pyblock

energy_array = np.array(energy_values_int)

reblock_data = pyblock.blocking.reblock(energy_array[:-10])
for reblock_iter in reblock_data:
    print(reblock_iter)

In [ ]:
df

In [10]:
import pandas as pd
import numpy as np
import statistics
import math
import pyblock

# Load data
df = pd.read_csv("results3_BACKUP240709.txt", sep=' ')

# from kcal/mol to ev/formate 
def apply_scaling(row):
    if '_full_' in row['model']:
        return row['energy'] * 0.0433641153087705 / 36
    elif '_half_' in row['model']:
        return row['energy'] * 0.0433641153087705 / 18
    else:
        return row['energy']

# Rename columns
df.columns = ['model', 'timestep', 'energy']

df['energy'] = df.apply(apply_scaling, axis=1)

# Filter models ending with *_300.0txt
df = df[df['model'].str.endswith('300.0.txt')]

# Convert timestep to nanoseconds
df['timestep'] = round(df['timestep'] * 0.5e-6, 1)

# Group by model and aggregate energy values into lists
df = df.groupby(['model']).agg(list).reset_index()

# Calculate mean and standard deviation for the last 50 energy values
df['mean_energy'] = df['energy'].apply(lambda x: statistics.mean(x[-50:]))
df['std_energy'] = df['energy'].apply(lambda x: statistics.stdev(x[-50:]))
df['variance'] = df['energy'].apply(lambda x: statistics.variance(x[-50:]))

# Values to be used in the loop
values = [9, 17, 25, 33, 41, 49, 57, 65]

# Initialize lists to store the results
models = []
mean_energy_differences = []
std_energy_differences = []

# Loop through the values and calculate the mean and standard deviation differences
for value in values:
    tet_full_model = f'{value}_tet_full_300.0.txt'
    int_full_model = f'{value}_int_full_300.0.txt'
    tet_half_model = f'{value}_tet_half_300.0.txt'
    int_half_model = f'{value}_int_half_300.0.txt'
    
    # Check if the models exist in the DataFrame before calculating the differences
    if tet_full_model in df['model'].values and int_full_model in df['model'].values:
        mean_energy_full_diff = ((df[df['model'] == tet_full_model]['mean_energy'].values[0] - 
                                  df[df['model'] == int_full_model]['mean_energy'].values[0]))
        
        std_full = ((math.sqrt((((df[df['model'] == tet_full_model]['std_energy'].values[0])**2)) + 
                               (((df[df['model'] == int_full_model]['std_energy'].values[0])**2)))))
        
        models.append(f'{tet_full_model} - {int_full_model}')
        
        mean_energy_differences.append(mean_energy_full_diff)
        std_energy_differences.append(std_full)
    
    if tet_half_model in df['model'].values and int_half_model in df['model'].values:
        mean_energy_half_diff = ((df[df['model'] == tet_half_model]['mean_energy'].values[0] - 
                                  df[df['model'] == int_half_model]['mean_energy'].values[0]))
        std_half = ((math.sqrt((((df[df['model'] == tet_half_model]['std_energy'].values[0])**2)) + 
                               (((df[df['model'] == int_half_model]['std_energy'].values[0])**2)))))
        
        models.append(f'{tet_half_model} - {int_half_model}')
        
        mean_energy_differences.append(mean_energy_half_diff)
        std_energy_differences.append(std_half)

# Create a new DataFrame with the results
diff_df = pd.DataFrame({
    'model': models,
    'mean_energy_difference': mean_energy_differences,
    'std_energy_difference': std_energy_differences
})

print(diff_df)

                                            model  mean_energy_difference  \
0     9_tet_full_300.0.txt - 9_int_full_300.0.txt                0.010600   
1     9_tet_half_300.0.txt - 9_int_half_300.0.txt                0.179190   
2   17_tet_full_300.0.txt - 17_int_full_300.0.txt               -0.008287   
3   17_tet_half_300.0.txt - 17_int_half_300.0.txt                0.151774   
4   25_tet_full_300.0.txt - 25_int_full_300.0.txt               -0.006794   
5   25_tet_half_300.0.txt - 25_int_half_300.0.txt                0.130911   
6   33_tet_full_300.0.txt - 33_int_full_300.0.txt               -0.016430   
7   33_tet_half_300.0.txt - 33_int_half_300.0.txt                0.191139   
8   41_tet_full_300.0.txt - 41_int_full_300.0.txt                0.017707   
9   41_tet_half_300.0.txt - 41_int_half_300.0.txt                0.173216   
10  49_tet_full_300.0.txt - 49_int_full_300.0.txt               -0.010191   
11  49_tet_half_300.0.txt - 49_int_half_300.0.txt                0.125178   

In [ ]:
df

In [ ]:
df['variance'] = df['energy'].apply(lambda x: statistics.variance(x[-50:]))

import math

print((math.sqrt((df[df['model'] == tet_full_model]['variance'].values[0])+(df[df['model'] == int_full_model]['variance'].values[0]))* 0.0433641153087705) / 36)

In [ ]:
                                            model  mean_energy_difference  \
0     9_tet_full_300.0.txt - 9_int_full_300.0.txt                0.010600   
1     9_tet_half_300.0.txt - 9_int_half_300.0.txt                0.179190   
2   17_tet_full_300.0.txt - 17_int_full_300.0.txt               -0.008287   
3   17_tet_half_300.0.txt - 17_int_half_300.0.txt                0.151774   
4   25_tet_full_300.0.txt - 25_int_full_300.0.txt               -0.006794   
5   25_tet_half_300.0.txt - 25_int_half_300.0.txt                0.130911   
6   33_tet_full_300.0.txt - 33_int_full_300.0.txt               -0.016430   
7   33_tet_half_300.0.txt - 33_int_half_300.0.txt                0.191139   
8   41_tet_half_300.0.txt - 41_int_half_300.0.txt                0.173216   
9   49_tet_full_300.0.txt - 49_int_full_300.0.txt               -0.010191   
10  49_tet_half_300.0.txt - 49_int_half_300.0.txt                0.125178   

    std_energy_difference  
0                0.008256  
1                0.014821  
2                0.009220  
3                0.020217  
4                0.012207  
5                0.022393  
6                0.011368  
7                0.025960  
8                0.030315  
9                0.017169  
10               0.039984  


model	energy_difference	mean_energy	std_energy
0	9_tet_full_300.0.txt - 9_int_full_300.0.txt	[0.0072273525514674475, 0.012045587585760131, ...	0.010600	0.007389
1	9_tet_half_300.0.txt - 9_int_half_300.0.txt	[0.16381999116646284, 0.1662291086836376, 0.19...	0.179190	0.014576
2	17_tet_full_300.0.txt - 17_int_full_300.0.txt	[-0.014454705102934895, 0.0024091175171747636,...	-0.008287	0.008753
3	17_tet_half_300.0.txt - 17_int_half_300.0.txt	[0.1517744035807027, 0.14695616854646687, 0.12...	0.151774	0.020174
4	25_tet_full_300.0.txt - 25_int_full_300.0.txt	[-0.019272940137284422, -0.02047749889578654, ...	-0.006794	0.013358
5	25_tet_half_300.0.txt - 25_int_half_300.0.txt	[0.13009234592630037, 0.13250146344353197, 0.1...	0.130911	0.023547
6	33_tet_full_300.0.txt - 33_int_full_300.0.txt	[-0.028909410205926633, -0.003613676275790567,...	-0.016430	0.011832
7	33_tet_half_300.0.txt - 33_int_half_300.0.txt	[0.15418352109782063, 0.17345646123510505, 0.1...	0.191139	0.025396
8	41_tet_half_300.0.txt - 41_int_half_300.0.txt	[0.17586557875233666, 0.21441145902667813, 0.1...	0.173216	0.031891
9	49_tet_full_300.0.txt - 49_int_full_300.0.txt	[-0.009636470068585368, -0.015659263861493855,...	-0.010191	0.017647
10	49_tet_half_300.0.txt - 49_int_half_300.0.txt	[0.2120023415095602, 0.17104734371787345, 0.14...	0.125178	0.039089

In [ ]:
import pandas as pd
import numpy as np
import statistics
import math
import pyblock


# Load data
df = pd.read_csv("results3.txt", sep=' ', header=None)

# Rename columns
df.columns = ['model', 'timestep', 'energy']

# Function to apply scaling based on model type
def apply_scaling(row):
    if '_full_' in row['model']:
        return row['energy'] * 0.0433641153087705 / 36
    elif '_half_' in row['model']:
        return row['energy'] * 0.0433641153087705 / 18
    else:
        return row['energy']

# Apply scaling
df['energy'] = df.apply(apply_scaling, axis=1)

# Filter models ending with *_300.0txt
df = df[df['model'].str.endswith('300.0.txt')]

# Convert timestep to nanoseconds
df['timestep'] = round(df['timestep'] * 0.5e-6, 1)

# Group by model and aggregate energy values into lists
df = df.groupby(['model']).agg(list).reset_index()

# Initialize lists to store the results
models = []
energy_differences = []

# Define values
values = [9, 17, 25, 33, 41, 49]

# Loop through the values and calculate the energy differences
for value in values:
    tet_full_model = f'{value}_tet_full_300.0.txt'
    int_full_model = f'{value}_int_full_300.0.txt'
    tet_half_model = f'{value}_tet_half_300.0.txt'
    int_half_model = f'{value}_int_half_300.0.txt'

    # Check if the models exist in the DataFrame before calculating the differences
    if tet_full_model in df['model'].values and int_full_model in df['model'].values:
        tet_full_energy = df[df['model'] == tet_full_model]['energy'].values[0]
        int_full_energy = df[df['model'] == int_full_model]['energy'].values[0]
        if len(tet_full_energy) == len(int_full_energy):
            # Subtract energy values directly if they have the same length
            energy_full_diff = [a - b for a, b in zip(tet_full_energy, int_full_energy)]
            models.append(f'{tet_full_model} - {int_full_model}')
            energy_differences.append(energy_full_diff)
        else:
            print(f"Length mismatch for {tet_full_model} and {int_full_model}")

    if tet_half_model in df['model'].values and int_half_model in df['model'].values:
        tet_half_energy = df[df['model'] == tet_half_model]['energy'].values[0]
        int_half_energy = df[df['model'] == int_half_model]['energy'].values[0]
        if len(tet_half_energy) == len(int_half_energy):
            # Subtract energy values directly if they have the same length
            energy_half_diff = [a - b for a, b in zip(tet_half_energy, int_half_energy)]
            models.append(f'{tet_half_model} - {int_half_model}')
            energy_differences.append(energy_half_diff)
        else:
            print(f"Length mismatch for {tet_half_model} and {int_half_model}")

# Create a new DataFrame with the results
diff_df = pd.DataFrame({
    'model': models,
    'energy_difference': energy_differences
})

print(diff_df)

In [ ]:
diff_df['mean_energy'] = diff_df['energy_difference'].apply(lambda x: statistics.mean(x[-50:]))
diff_df['std_energy'] = diff_df['energy_difference'].apply(lambda x: statistics.stdev(x[-50:]))

In [ ]:
diff_df

In [ ]:
diff_df['mean_energy'] = diff_df['energy_difference'].apply(lambda x: statistics.mean(x))
diff_df['std_energy'] = diff_df['energy_difference'].apply(lambda x: statistics.stdev(x))

diff_df

In [ ]:

# Initialize lists to store the results
models = []
energy_diff = []
std_energy_differences = []

# Loop through the values and calculate the mean and standard deviation differences
for value in values:
    tet_full_model = f'{value}_tet_full_300.0.txt'
    int_full_model = f'{value}_int_full_300.0.txt'
    tet_half_model = f'{value}_tet_half_300.0.txt'
    int_half_model = f'{value}_int_half_300.0.txt'


    # Check if the models exist in the DataFrame before calculating the differences
    if tet_full_model in df['model'].values and int_full_model in df['model'].values:


        energy_diff_full = (df[df['model'] == tet_full_model]['energy'] - df[df['model'] == int_full_model]['energy'])
        print (df[df['model'] == tet_full_model]['energy'] )
        

In [ ]:
models.append(f'{tet_full_model} - {int_full_model}')        
        energy_diff.append(energy_diff_full)
    
    if tet_half_model in df['model'].values and int_half_model in df['model'].values:
        energy_diff_half = (df[df['model'] == tet_half_model]['mean_energy'].values[0] - df[df['model'] == int_half_model]['mean_energy'].values[0])
        models.append(f'{tet_half_model} - {int_half_model}')
        energy_diff.append(energy_diff_full)
     
# Create a new DataFrame with the results
diff_df = pd.DataFrame({
    'model': models,
    'energy_difference': energy_diff
})

print(diff_df)